In [1]:
import pandas as pd

# Load the CSV
df = pd.read_csv("results_safe.csv")

# Ensure findings is string type
df['findings'] = df['findings'].astype(str)

# Define the variations you want to match
reentrancy_variants = ["Reentrancy", "Re_Entrancy", "reentrancy", "REENTRANCY", "State_access_after_external_call_SWC_107"]

# Check if any variant is in the findings string
df['detected_reentrancy'] = df['findings'].apply(
    lambda x: any(variant in x for variant in reentrancy_variants)
)


# Filter rows where reentrancy was detected
reentrancy_df = df[df['detected_reentrancy']]

# Group by basename and aggregate tool names
reentrancy_summary = (
    reentrancy_df
    .groupby('basename')
    .agg(
        tools_detected_reentrancy=('toolid', lambda x: sorted(set(x))),
        num_tools_detected_reentrancy=('toolid', lambda x: x.nunique())
    )
    .reset_index()
)

# Optional: include all contracts and fill 0/tools if no detection
all_contracts = df[['basename']].drop_duplicates()
result = all_contracts.merge(reentrancy_summary, on='basename', how='left')
result['num_tools_detected_reentrancy'] = result['num_tools_detected_reentrancy'].fillna(0).astype(int)
result['tools_detected_reentrancy'] = result['tools_detected_reentrancy'].apply(lambda x: x if isinstance(x, list) else [])

# Print the result
result


,basename,tools_detected_reentrancy,num_tools_detected_reentrancy
0,01dd4a2eacf1f751bcf9070bca20c3a2_cgt.sol,"[conkas, mythril-0.24.7, osiris, oyente, sfuzz...",6
1,04045504816e9b4137a6838c8a53dde4_cgt.sol,"[conkas, osiris, oyente, sfuzz]",4
2,083e60356eeecf6edf0e82b79f66a4ba_cgt.sol,"[conkas, sfuzz]",2
3,0a7dcd7ad58d91e0373b21e4e4522cdf_cgt.sol,"[confuzzius, osiris]",2
4,0bf0ebe31b1e67618bc28a9d3738a2e4.sol,[conkas],1
...,...,...,...
307,ec7ad4af1fc95654260069e7932dbf3d_cgt.sol,"[conkas, slither-0.10.4]",2
308,eebbd1de0bd005402b5dde75513c45f6_cgt.sol,[sfuzz],1
309,eeec1e8a9be3b58748c8e9f28342e429_cgt.sol,"[conkas, sfuzz, slither-0.10.4, solhint-3.3.8]",4
310,f21a824407dfb37c81f7fbbd8d8e0e58_cgt.sol,"[conkas, sfuzz, slither-0.10.4]",3


In [2]:
pd.set_option('display.max_rows', None)  # Show all rows
result = result.sort_values(by='num_tools_detected_reentrancy', ascending=False)
result

,basename,tools_detected_reentrancy,num_tools_detected_reentrancy
0,01dd4a2eacf1f751bcf9070bca20c3a2_cgt.sol,"[conkas, mythril-0.24.7, osiris, oyente, sfuzz...",6
297,36ee840ee2c730b0dd466462fb94221d_cgt.sol,"[conkas, mythril-0.24.7, osiris, oyente, sfuzz...",6
298,39afddc46dfaa99f28c85e727921bdae_cgt.sol,"[conkas, mythril-0.24.7, osiris, oyente, sfuzz...",6
1,04045504816e9b4137a6838c8a53dde4_cgt.sol,"[conkas, osiris, oyente, sfuzz]",4
309,eeec1e8a9be3b58748c8e9f28342e429_cgt.sol,"[conkas, sfuzz, slither-0.10.4, solhint-3.3.8]",4
304,ccaaeeb118de7427945743ed5e655e8d_cgt.sol,"[confuzzius, conkas, osiris, slither-0.10.4]",4
29,0x0b5da756938e334c97ce20715e32a4a8fea12ba9.sol,"[osiris, slither-0.10.4, solhint-3.3.8]",3
284,0xf6a1fa796cfbdcc4adb08dbf634438fb35561d8d_rs.sol,"[conkas, osiris, sfuzz]",3
279,0xf45ae69cca1b9b043dae2c83a5b65bc605bec5f5.sol,"[mythril-0.24.7, sfuzz, slither-0.10.4]",3
257,0xde1586fb826fbd11bedb1b59e76d9e1b9e69e3ca.sol,"[conkas, sfuzz, slither-0.10.4]",3
